In [71]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<<article class="day-desc"><h2>--- Day 13: Knights of the Dinner Table ---</h2><p>In years past, the holiday feast with your family hasn't gone so well.  Not everyone gets along!  This year, you resolve, will be different.  You're going to find the <em>optimal seating arrangement</em> and avoid all those awkward conversations.</p>
<p>You start by writing up a list of everyone invited and the amount their happiness would increase or decrease if they were to find themselves sitting next to each other person.  You have a circular table that will be just big enough to fit everyone comfortably, and so each person will have exactly two neighbors.</p>
<p>For example, suppose you have only four attendees planned, and you <span title="Finding a method to calculate happiness units is left as an exercise for the reader.">calculate</span> their potential happiness as follows:</p>
<pre><code>Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 79 happiness units by sitting next to Carol.
Alice would lose 2 happiness units by sitting next to David.
Bob would gain 83 happiness units by sitting next to Alice.
Bob would lose 7 happiness units by sitting next to Carol.
Bob would lose 63 happiness units by sitting next to David.
Carol would lose 62 happiness units by sitting next to Alice.
Carol would gain 60 happiness units by sitting next to Bob.
Carol would gain 55 happiness units by sitting next to David.
David would gain 46 happiness units by sitting next to Alice.
David would lose 7 happiness units by sitting next to Bob.
David would gain 41 happiness units by sitting next to Carol.
</code></pre>
<p>Then, if you seat Alice next to David, Alice would lose <code>2</code> happiness units (because David talks so much), but David would gain <code>46</code> happiness units (because Alice is such a good listener), for a total change of <code>44</code>.</p>
<p>If you continue around the table, you could then seat Bob next to Alice (Bob gains <code>83</code>, Alice gains <code>54</code>).  Finally, seat Carol, who sits next to Bob (Carol gains <code>60</code>, Bob loses <code>7</code>) and David (Carol gains <code>55</code>, David gains <code>41</code>).  The arrangement looks like this:</p>
<pre><code>     +41 +46
+55   David    -2
Carol       Alice
+60    Bob    +54
     -7  +83
</code></pre>
<p>After trying every other seating arrangement in this hypothetical scenario, you find that this one is the most optimal, with a total change in happiness of <code>330</code>.</p>
<p>What is the <em>total change in happiness</em> for the optimal seating arrangement of the actual guest list?</p>
</article>


In [72]:
s = """
Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 79 happiness units by sitting next to Carol.
Alice would lose 2 happiness units by sitting next to David.
Bob would gain 83 happiness units by sitting next to Alice.
Bob would lose 7 happiness units by sitting next to Carol.
Bob would lose 63 happiness units by sitting next to David.
Carol would lose 62 happiness units by sitting next to Alice.
Carol would gain 60 happiness units by sitting next to Bob.
Carol would gain 55 happiness units by sitting next to David.
David would gain 46 happiness units by sitting next to Alice.
David would lose 7 happiness units by sitting next to Bob.
David would gain 41 happiness units by sitting next to Carol.
"""


def parse(s: str) -> defaultdict[defaultdict[int]]:
    preferred = defaultdict(lambda: defaultdict(int))
    for l in s.strip().splitlines():
        person, _, gain, number, *_, neigbor = l[:-1].split(" ")
        gain = int(number) if gain == "gain" else -int(number)
        preferred[person][neigbor] = gain
    return preferred


def add_myself(preferred: defaultdict[defaultdict[int]]) -> None:
    for other in list(preferred.keys()):
        preferred[other]["myself"] = 0
        preferred["myself"][other] = 0


def optimal_seating_arrangement(s: str, do_add_myself: bool = False) -> int:
    def dfs(left: str, right: str) -> int:
        if not not_seen:
            max_happiness = preferred[left][right] + preferred[right][left]
        elif len(not_seen) == 1:
            last = not_seen.pop()
            max_happiness = preferred[left][last] + preferred[last][left]
            max_happiness += preferred[last][right] + preferred[right][last]
            not_seen.add(last)
        else:
            max_happiness = -inf
            for ll, rr in permutations(list(not_seen), 2):
                not_seen.remove(ll)
                not_seen.remove(rr)
                h = preferred[left][ll] + preferred[ll][left]
                h += preferred[rr][right] + preferred[right][rr]
                h += dfs(ll, rr)
                if h > max_happiness:
                    max_happiness = h
                not_seen.add(rr)
                not_seen.add(ll)
        return max_happiness

    preferred = parse(s)
    if do_add_myself:
        add_myself(preferred)

    not_seen = set(preferred.keys())
    first = not_seen.pop()
    return dfs(first, first)


optimal_seating_arrangement(s)

330

In [73]:
with open("../input/day13.txt") as f:
    print(optimal_seating_arrangement(f.read()))

733


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>733</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>In all the commotion, you realize that you forgot to seat yourself.  At this point, you're pretty apathetic toward the whole thing, and your happiness wouldn't really go up or down regardless of who you sit next to.  You assume everyone else would be just as ambivalent about sitting next to you, too.</p>
<p>So, add yourself to the list, and give all happiness relationships that involve you a score of <code>0</code>.</p>
<p>What is the <em>total change in happiness</em> for the optimal seating arrangement that actually includes yourself?</p>
</article>

</main>


In [74]:
optimal_seating_arrangement(s, do_add_myself=True)

286

In [75]:
with open("../input/day13.txt") as f:
    print(optimal_seating_arrangement(f.read(), do_add_myself=True))

725


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>725</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="13/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Knights+of+the+Dinner+Table%22+%2D+Day+13+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F13&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Knights+of+the+Dinner+Table%22+%2D+Day+13+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F13';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
